In [5]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import os

In [6]:
base_dir = os.path.abspath(os.path.join("..", "data", "raw"))
file_path = os.path.join(base_dir, "train.csv") # initial raw dataset
patients = pd.read_csv(file_path, low_memory=False)

patients.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


## Step 1
### Observations on data
- <strong>Inconsistency:</strong> different time intervals for different patients in the dataset (some recorded every 5 minutes and others every 15 minutes)
- <strong>Potential data loss:</strong> if we attempt to standardize the time intervals, we risk losing data or oversimplifying trends, especially in patients with finer granularity (5-minute intervals).

<b>Possible Approaches:</b>

<b>1.</b> Resampling the data to a consistent time interval:
- Up-sampling (resample to 5 minutes for all patients).
- Down-sampling (resample to 15 minutes for all patients).

<b>a) Up-sampling:</b> involves resampling all patient data to the finer 5-minute intervals. For patients who have data recorded every 15 minutes, we could interpolate missing values for the in-between time points.

<b>Pros:</b>
- Keeps the finer-granularity data intact for those patients who already have 5-minute intervals.
- Allows for more detailed time-series analysis.
  
<b>Cons:</b>
- Interpolating data for the patients who originally have 15-minute intervals might introduce artificial data points, which may not capture the true variability of the measurements.

<b>b) Down-sampling:</b> resampling everyone’s data to 15-minute intervals by aggregating the 5-minute data for patients with finer granularity (e.g. taking averages or sums over each 15-minute period).

<b>Pros:</b>
- Simpler, and no interpolation is needed.
- Keeps the data more consistent with what was actually measured for those with 15-minute intervals.

<b>Cons:</b>
- We might lose detail from patients who had more frequent measurements, which could result in losing important patterns in their data.

<b>2.</b> Handling each interval separately: another option for building models on patients with 5-min interval separately from the patients with 15-min interval.

<b>Pros:</b>
- We retain the original data for each patient.
- We don’t need to interpolate or downsample keeping the true variability.

<b>Cons:</b>
- This requires more work since we’re essentially running two analyses or models, one for each group.
- It leads to smaller training sets for each group.

<b>3.</b> Feature Engineering: we can create additional feature to account for differences in time intervals:
- Time interval: indicating whether the data point comes from a 5-min or 15-min interval.

This will let the model account for different time resolutions without explicitly resampling the data.

<b>Recommended approach:</b> Based on the fact that we have only 3 patients with a 15-minute interval and 6 patient with a 5-minute interval, it is reasonable to resample the dataset to 5-minute intervals for all patients. Steps:
- Resample the data for all patients to 5-minute intervals.
- Interpolate the missing data points for patients who originally had 15-minute intervals.



&nbsp;&nbsp;&nbsp;&nbsp;In the following, we introduce an additional variable <code>'pseudo_datetime'</code> that will be used for the calculation of the time interval category (5- or 15 min). This might be usefull if we want to do an analisys on both groups separately.

In [7]:
# Step 1: Define a fake start date (e.g., January 1, 2020)
fake_start_date = '2020-01-01'

# Step 2: Convert 'time' to timedelta (duration since midnight)
patients['time_delta'] = pd.to_timedelta(patients['time'])

# Step 3: For each patient, start the datetime from the same fake start date
def create_pseudo_datetime(group):
    # Start from the same fake date
    group['pseudo_datetime'] = pd.to_datetime(fake_start_date) + group['time_delta']
    
    # Calculate the days elapsed when time resets (new day)
    group['days_elapsed'] = group['time_delta'].diff().apply(lambda x: 1 if x < pd.Timedelta(0) else 0).cumsum()
    
    # Add days_elapsed to the pseudo_datetime to simulate continuous time
    group['pseudo_datetime'] = group['pseudo_datetime'] + pd.to_timedelta(group['days_elapsed'], unit='D')
    
    return group

# Step 4: Apply the function to each patient group and reset the index
patients = patients.groupby('p_num', group_keys=False).apply(create_pseudo_datetime).reset_index(drop=True)

# Step 5: Calculate time intervals
patients['time_interval'] = patients.groupby('p_num')['pseudo_datetime'].diff().dt.total_seconds() / 60




C:\Users\svetl\AppData\Local\Temp\ipykernel_16848\286818186.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  patients = patients.groupby('p_num', group_keys=False).apply(create_pseudo_datetime).reset_index(drop=True)


In [10]:
patients['time_interval'].value_counts()

time_interval
5.0      151756
15.0      24527
20.0        403
10.0        121
30.0         33
          ...  
550.0         1
800.0         1
805.0         1
945.0         1
295.0         1
Name: count, Length: 69, dtype: int64

In [11]:
# Identify the indices where time_interval is 20.0
time_20_idx = patients[patients['time_interval'] == 945.0].index

# Include the previous record for each identified row
# Combine the current and previous indices, and drop duplicates in case of overlaps
previous_and_current_idx = time_20_idx.union(time_20_idx - 1)

# Filter the DataFrame based on these indices
result = patients.loc[previous_and_current_idx]

# Display the result
print(result[['p_num', 'time', 'pseudo_datetime', 'time_interval']])


       p_num      time     pseudo_datetime  time_interval
174124   p12  01:50:00 2020-03-22 01:50:00            5.0
174125   p12  17:35:00 2020-03-22 17:35:00          945.0


&nbsp;&nbsp;&nbsp;&nbsp;We can see that some time records are not consistent. A reason might be that the continuous glucose monitor (CGM) or other devices might have been turned off and on, causing a "jump" in the recorded time.
<p>We'll replace <code>'time_interval'</code> with its Mode (most frequent value) per Patient.</p>

In [73]:
# function to calculate the mode and replace the column
def replace_with_mode(group):
    mode_value = group['time_interval'].mode()[0]  
    group['time_interval'] = mode_value            
    return group

# Apply this function to each patient group
patients = patients.groupby('p_num').apply(replace_with_mode)


C:\Users\svetl\AppData\Local\Temp\ipykernel_25160\2001115506.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  patients = patients.groupby('p_num').apply(replace_with_mode)


In [75]:
#patients.groupby('p_num')['time_interval'].apply(lambda x: x.mode()[0])

In [74]:
patients['time_interval'].value_counts()

time_interval
5.0     151894
15.0     25130
Name: count, dtype: int64

In [76]:
patients.drop(['time_delta', 'days_elapsed'], axis=1)

id p_num      time  bg-5:55  bg-5:50  bg-5:45  bg-5:40  \
p_num                                                                         
p01   0           p01_0   p01  06:10:00      NaN      NaN      9.6      NaN   
      1           p01_1   p01  06:25:00      NaN      NaN      9.7      NaN   
      2           p01_2   p01  06:40:00      NaN      NaN      9.2      NaN   
      3           p01_3   p01  06:55:00      NaN      NaN      8.7      NaN   
      4           p01_4   p01  07:10:00      NaN      NaN      8.4      NaN   
...                 ...   ...       ...      ...      ...      ...      ...   
p12   177019  p12_25294   p12  23:35:00      8.8      9.1      9.2      9.4   
      177020  p12_25295   p12  23:40:00      9.1      9.2      9.4      9.8   
      177021  p12_25296   p12  23:45:00      9.2      9.4      9.8     10.2   
      177022  p12_25297   p12  23:50:00      9.4      9.8     10.2     10.4   
      177023  p12_25298   p12  23:55:00      9.8     10.2     10.4     10.3   

              bg-5:35  bg-5:30  bg-5:25  bg-5:20  bg-5:15  bg-5:10  bg-5:05  \
p_num                                                                         
p01   0           NaN      9.7      NaN      NaN      9.2      NaN      NaN   
      1           NaN      9.2      NaN      NaN      8.7      NaN      NaN   
      2           NaN      8.7      NaN      NaN      8.4      NaN      NaN   
      3           NaN      8.4      NaN      NaN      8.1      NaN      NaN   
      4           NaN      8.1      NaN      NaN      8.3      NaN      NaN   
...               ...      ...      ...      ...      ...      ...      ...   
p12   177019      9.8     10.2     10.4     10.3     10.1     10.0      9.8   
      177020     10.2     10.4     10.3     10.1     10.0      9.8      9.5   
      177021     10.4     10.3     10.1     10.0      9.8      9.5      9.2   
      177022     10.3     10.1     10.0      9.8      9.5      9.2      9.0   
      177023     10.1     10.0      9.8      9.5      9.2      9.0      8.9   

              bg-5:00  bg-4:55  bg-4:50  bg-4:45  bg-4:40  bg-4:35  bg-4:30  \
p_num                                                                         
p01   0           8.7      NaN      NaN      8.4      NaN      NaN      8.1   
      1           8.4      NaN      NaN      8.1      NaN      NaN      8.3   
      2           8.1      NaN      NaN      8.3      NaN      NaN      9.6   
      3           8.3      NaN      NaN      9.6      NaN      NaN     11.1   
      4           9.6      NaN      NaN     11.1      NaN      NaN     11.8   
...               ...      ...      ...      ...      ...      ...      ...   
p12   177019      9.5      9.2      9.0      8.9      8.7      8.3      7.8   
      177020      9.2      9.0      8.9      8.7      8.3      7.8      7.6   
      177021      9.0      8.9      8.7      8.3      7.8      7.6      7.6   
      177022      8.9      8.7      8.3      7.8      7.6      7.6      7.7   
      177023      8.7      8.3      7.8      7.6      7.6      7.7      7.7   

              bg-4:25  bg-4:20  bg-4:15  bg-4:10  bg-4:05  bg-4:00  bg-3:55  \
p_num                                                                         
p01   0           NaN      NaN      8.3      NaN      NaN      9.6      NaN   
      1           NaN      NaN      9.6      NaN      NaN     11.1      NaN   
      2           NaN      NaN     11.1      NaN      NaN     11.8      NaN   
      3           NaN      NaN     11.8      NaN      NaN     12.8      NaN   
      4           NaN      NaN     12.8      NaN      NaN     13.9      NaN   
...               ...      ...      ...      ...      ...      ...      ...   
p12   177019      7.6      7.6      7.7      7.7      7.7      7.6      7.7   
      177020      7.6      7.7      7.7      7.7      7.6      7.7      7.8   
      177021      7.7      7.7      7.7      7.6      7.7      7.8      7.9   
      177022      7.7      7.7      7.6      7.7      7.8      7.9  

## Challenges and potential difficulties
- <b>Missing Data</b>
    - In the dataset, many columns (e.g. carbs-\*, activity-\*) had high percentages of missing values. This is a common issue in studies involving self-reported data or continuous monitoring (device resets etc.).    
    - Handling strategy: impute missing values, use of statistical models to fill in gaps <font color="red">(?)</font>
- <b>Outliers</b>
    - From the variable analysis, we noticed a large number of outliers. Skewness in insulin doses or other features might be the cause of the detecttion of too many outliers. Here, IQR method was used.
    - Handling strategy: consider transformations to reduce skewness before detecting outliers  <font color="red">(?)</font>
- <b>Non-normal distributions</b>
    - Features like insulin and blood glucose are often highly skewed. This could impact the performance of models that assume normality or require scaling.
- <b>Feature engineering for Time-Series</b>
    - Extracting useful features such as:
        - Lagged features: past glucose, insulin, carb or other values from previous time steps.
        - Rolling averages/windows: smoothed glucose or insulin averages over time to capture trends.  <font color="red">(?)</font>
- <b>Multicollinearity</b>
    - Since the dataset contains many related measurements over time, there might be strong correlations between features. This could impact model performance, especially for regression models.
    - Handling strategy: regularization techniques (Lasso, Ridge) or dimensionality reduction (PCA).  <font color="red">(?)</font>


## Possible biases
- <b>Selection Bias:</b>
    - If the study only included certain types of participants (e.g. individuals with a specific health condition, age group, or gender), this could lead to selection bias, meaning the results may not generalize to other populations.
- <b>Measurement Bias:</b>
    - Self-reported data like carbohydrate intake and activity are prone to inaccuracies, which can lead to measurement bias. Wearable devices (for heart rate, steps, etc.) can also be less reliable depending on usage conditions.
- <b>Sampling Bias:</b>
    - If the self-reported measurements are not taken consistently (e.g., some participants report their data more frequently than others), the dataset might not represent all participants equally, leading to biased conclusions.
